In [ ]:
def tokenizar(expresion):
    tokens = []
    tmp = ''
    for t in expresion:
        if t.isdigit() or t == '.' or t.isalpha():
            tmp += t
        else:
            if tmp:
                tokens.append(float(tmp) if tmp.isnumeric() else tmp.lower())
                tmp = ''
            if t.strip():
                tokens.append(t)
    if tmp: 
        tokens.append(float(tmp) if tmp.isnumeric() else tmp.lower())

    return tokens

class Nodo:
    def evaluar(self, variables):
        pass
    
    def variables(self):
        return set()

class Numero(Nodo):
    def __init__(self, valor):
        self.valor = valor

    def evaluar(self, variables):
        return self.valor
    
    def __str__(self):
        return str(self.valor)

class Variable(Nodo):
    def __init__(self, nombre):
        self.nombre = nombre

    def evaluar(self, variables):
        return variables[self.nombre]
    
    def variables(self):
        return set(self.nombre)
    
    def __str__(self):
        return self.nombre

class Operacion(Nodo):
    def __init__(self, op, izq, der):
        self.op   = op
        self.izq = izq
        self.der = der

    def evaluar(self, variables):
        izq = self.izq.evaluar(variables)
        der = self.der.evaluar(variables)
        match self.op: 
            case '+':
                return izq + der
            case '-':
                return izq - der
            case '*':
                return izq * der
            case '/':
                return izq / der
            case '^':
                return izq ** der

    def variables(self):
        return self.izq.variables() | self.der.variables()
    
    def __str__(self):
        return f"({self.izq} {self.op} {self.der})"

def compilar(tokens):
    """Convierte una lista plana de tokens en una lista anidada que refleja la precedencia de operadores."""
    # Definir la precedencia de los operadores
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}

    def token():
        return tokens[0] if tokens else None

    def avanzar():
        if not tokens: return None 
        return tokens.pop(0)

    def convertir(minimo=1):

        if token() == '(':
            avanzar()
            izq = convertir()
            if avanzar() != ')': raise ValueError("Se esperaba ')'")
        elif isinstance(token(),  float):
            izq = Numero(avanzar())
        elif isinstance(token(), str):
            izq = Variable(avanzar())
        else:
            raise ValueError(f"Token inesperado: {token()}")

        while precedence.get(token(), 0) >= minimo:
            op = avanzar()  
            der = convertir(precedence[op])
            izq = Operacion(op, izq, der)

        return izq

    resultado = convertir()
    if tokens: raise ValueError(f"Tokens restantes: {tokens}")
    
    return resultado

# Ejemplo de uso
if __name__ == "__main__":
    # Expresión de ejemplo como lista de tokens
    tokens = tokenizar("10 + x * (50 + 1 * y) - 3")

    # Convertir la lista de tokens en una estructura anidada
    e = compilar(tokens)

    print("Estructura anidada de la expresión:")
    print(e)
    print("Variables en la expresión:", e.variables())
    print(e.evaluar({'x': 5, 'y': 3}))
    # Salida esperada:
    # [10, '+', [20, '*', [50, '+', [1, '*', 2]]], '-', 3]

    # Otro ejemplo con exponentes y asociatividad derecha
    tokens2 = [2, '^', 3, '^', 2]
    nested2 = compilar(tokens2)
    print("\nEstructura anidada de la expresión con exponentes:")
    print(nested2)
    # Salida esperada:
    # [2, '^', [3, '^', 2]]


In [41]:
class Expresion:
    contexto = {}
    tokens = []

    @staticmethod
    def tokenizar(expresion):
        tokens = []
        tmp = ''
        for t in expresion:
            if t.isdigit() or t == '.' or t.isalpha():
                tmp += t
            else:
                if tmp:
                    tokens.append(tmp.lower())
                    tmp = ''
                if t.strip():
                    tokens.append(t)
        if tmp: 
            tokens.append(tmp.lower())

        return tokens

    class Numero:
        def __init__(self, valor):
            self.valor = valor

        def evaluar(self):
            return self.valor

        def __str__(self):
            return f"n({self.valor})"

        def variables(self):
            return set()
        
    class Variable:
        def __init__(self, nombre):
            self.nombre = nombre

        def evaluar(self):
            return Expresion.contexto[self.nombre]

        def __str__(self):
            return f"v({self.nombre})"
        
        def variables(self):
            return set(self.nombre)

    class Operacion:
        def __init__(self, op, izq, der):
            self.op = op
            self.izq = izq
            self.der = der

        def evaluar(self):
            if self.op == '+':
                return self.izq.evaluar() + self.der.evaluar()
            elif self.op == '-':
                return self.izq.evaluar() - self.der.evaluar()
            elif self.op == '*':
                return self.izq.evaluar() * self.der.evaluar()
            elif self.op == '/':
                return self.izq.evaluar() / self.der.evaluar()
            elif self.op == '^':
                return self.izq.evaluar() ** self.der.evaluar()
            else:
                raise ValueError(f"Operador desconocido: {self.op}")

        def variables(self):
            return self.izq.variables() | self.der.variables()

        def __str__(self):
            return f"o({self.izq} {self.op} {self.der})"

    
    def __init__(self, expresion):
        """Convierte una lista plana de tokens en una lista anidada que refleja la precedencia de operadores."""
    
        precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}

        def token():
            return tokens[0] if tokens else None

        def avanzar():
            if not tokens: return None 
            return tokens.pop(0)

        def convertir(minimo=1):
            if token() == '(':
                avanzar()
                izq = convertir()
                if avanzar() != ')': raise ValueError("Se esperaba ')'")
            elif token().isdigit():
                izq = Expresion.Numero(float(avanzar()))
            elif token().isalpha():
                izq = Expresion.Variable(avanzar())
            else:
                raise ValueError(f"Token inesperado: {token()}")

            while precedence.get(token(), 0) >= minimo:
                op = avanzar()  
                der = convertir(precedence[op])
                izq = Expresion.Operacion(op, izq, der)

            return izq

        
        tokens.extend(Expresion.tokenizar(expresion))

        self.expresion = convertir()
        if tokens: raise ValueError(f"Tokens restantes: {tokens}")

        self.variables = self.expresion.variables()

    def evaluar(self, **contexto):
        if contexto:
            Expresion.contexto = contexto
        return self.expresion.evaluar()
    
# Ejemplo de uso
if __name__ == "__main__":
    # Definir el contexto con los valores de las variables

    # Expresión de ejemplo como lista de tokens
    tokens = ["10", "+", "x", "*", "(", "50", "+", "1", "*", "y", ")", "-", "3"]

    # Convertir la lista de tokens en una estructura anidada
    e = Expresion("10+x*(50+2*y)-3")

    print(e)
    print(e.variables)

    # Evaluar la expresión
    # resultado = e.evaluar(x=10, y=20)
    # print(f"Resultado: {resultado}")

ValueError: Tokens restantes: ['10', '+', 'x', '*', '(', '50', '+', '2', '*', 'y', ')', '-', '3']

In [38]:
"al".isalpha()

True